## Загрузим нужные библиотеки

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import classification_report
import math
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
%matplotlib inline


В этом примере мы попробуем предсказать итоговую оценку участинка только на основе **его действий**

In [2]:
df_train = pd.read_csv("train.csv")
df_user = pd.read_csv("user_decision.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

## Рассмотрим датасет по ближе

In [ ]:
df_train.shape


In [ ]:
df_train.info()

In [ ]:
df_train.head(30)

In [ ]:
df_user.shape

In [ ]:
df_user.head(3)

In [ ]:
df_user.info()

Заметим что в есть категория где пользватель совершал больше действий

In [ ]:
df_user.period.value_counts()

Пострим на распределение данных

In [ ]:
sns.countplot(x = "period" , data  = df_user).set_title('Распределение периода')

In [ ]:
df_user["decision_id"].hist(bins = 300).set_title('Распределение действий')

### Добавим новых значений

Узнаем частоту команд участника в каждом периоде

In [ ]:
df_user_train = df_user[df_user.user_id.map(lambda x:x in df_train.id.values)]

In [ ]:
df_decision_count = df_user_train.groupby(by=["user_id",	"period"]).count()

In [ ]:
df_decision_count.head(5)

In [ ]:
for id in df_train.id.values:
  for period in range(1,5):
    try :
      index = df_train[df_train.id == id].index[0]
      df_train.loc[index,"period_"+str(period)] = df_decision_count.loc[pd.IndexSlice[id, period:period], :].values[0][0]
    except IndexError:
      continue

In [ ]:
df_train.head(3)

In [ ]:
df_train=df_train.fillna(df_train.mean())

In [ ]:
df_train.isna().sum()

In [ ]:
df_train['sum']=df_train['period_1']+df_train['period_2']+df_train['period_3']+df_train['period_4']
df_train['quad']=df_train['period_1'] * df_train['period_1'] + df_train['period_2'] * df_train['period_2'] + df_train['period_3'] * df_train['period_3'] + df_train['period_4'] * df_train['period_4']
df_train['medan']=(df_train['period_1']+df_train['period_2']+df_train['period_3']+df_train['period_4'])/4
df_train['sqr_median']=np.sqrt(df_train['medan'])
df_train['sqr_sum']=np.sqrt(df_train['sum'])

In [ ]:
df_train

In [ ]:
df_train=df_train.append(df_train)

In [ ]:

df_train['Focus'].value_counts()


focus1=df_train.query("Adaptability==2").head()
focus2=df_train.query("Adaptability==1").head()
focus3=df_train.query("Adaptability==6").head()
focus4=df_train.query("Adaptability==3").head()
for i in range(1,10):
    df_train=df_train.append(focus1, ignore_index=True)
    df_train=df_train.append(focus2, ignore_index=True)
    df_train=df_train.append(focus3, ignore_index=True)
    df_train=df_train.append(focus4, ignore_index=True)

In [ ]:
df_train['Adaptability'].value_counts()

In [ ]:
df_train=df_train.query('Adaptability!=2')
df_train=df_train.query('Focus!=2')

In [ ]:
plt.rcParams['figure.figsize']=(15,15)

g = sns.heatmap(df_train.corr(), square = True, annot=True)

In [ ]:
df_train

In [ ]:
df_train['Focus'].value_counts()

In [ ]:
df_train['Adaptability'].value_counts()

focus2=df_train.query("Adaptability==1").head(25)
for i in range(1,10):
    df_train=df_train.append(focus2, ignore_index=True)
    

focus2=df_train.query("Focus==1").head(25)
focus3=df_train.query("Focus==6").head(25)
for i in range(1,5):
    df_train=df_train.append(focus2, ignore_index=True)
    df_train=df_train.append(focus3, ignore_index=True)

## Выделим выборки

In [ ]:
X = df_train.drop(['Analytical thinking','Systemic thinking','Adaptability', 'Focus','id'], axis = 1)
y = df_train['Analytical thinking']


In [ ]:
X.shape

In [ ]:
X

In [ ]:
X.isna().sum()

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение модели

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
rf_model_Analytical_thinking = GaussianNB() # Ваш код здесь
# обучим модель случайного леса
rf_model_Analytical_thinking.fit(X_train, y_train)

## Оценка точности

In [ ]:
pred = rf_model_Analytical_thinking.predict(X_test)

In [ ]:
pred

In [ ]:

print (classification_report(y_test, pred))

In [ ]:
X = df_train.drop(['Systemic thinking','Adaptability', 'Focus','id'], axis = 1)
y = df_train['Systemic thinking']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model_Systemic_thinking =  GaussianNB() # Ваш код здесь
# обучим модель случайного леса
rf_model_Systemic_thinking.fit(X_train, y_train)
pred = rf_model_Systemic_thinking.predict(X_test)
print (classification_report(y_test, pred))

In [ ]:
X = df_train.drop(['Adaptability', 'Focus','id'], axis = 1)
y = df_train['Adaptability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model_Adaptability =  GaussianNB() # Ваш код здесь
# обучим модель случайного леса
rf_model_Adaptability.fit(X_train, y_train)
pred = rf_model_Adaptability.predict(X_test)
print (classification_report(y_test, pred))

In [ ]:
X = df_train.drop(['Focus','id'], axis = 1)
y = df_train['Focus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model_Focus =  GaussianNB() # Ваш код здесь
# обучим модель случайного леса
rf_model_Focus.fit(X_train, y_train)
pred = rf_model_Focus.predict(X_test)
print (classification_report(y_test, pred))

In [ ]:
# Предикт.

In [ ]:
datapre=pd.read_csv('sample_solution.csv')

In [ ]:
datapre

In [ ]:
df_user_train = df_user[df_user.user_id.map(lambda x:x in datapre.id.values)]

In [ ]:
df_decision_count = df_user_train.groupby(by=["user_id",	"period"]).count()

In [ ]:
df_decision_count.head(5)

In [ ]:
for id in datapre.id.values:
  for period in range(1,5):
    try :
      index = datapre[datapre.id == id].index[0]
      datapre.loc[index,"period_"+str(period)] = df_decision_count.loc[pd.IndexSlice[id, period:period], :].values[0][0]
    except IndexError:
      continue

In [ ]:
datapre

In [ ]:
datapre=datapre.fillna(datapre.mean())
datapre.to_csv('dataprefil.csv',index=False)

In [ ]:
datapre['sum']=datapre['period_1']+datapre['period_2']+datapre['period_3']+datapre['period_4']
datapre['quad']=datapre['period_1'] * datapre['period_1'] + datapre['period_2'] * datapre['period_2'] + datapre['period_3'] * datapre['period_3'] + datapre['period_4'] * datapre['period_4']
datapre['medan']=(datapre['period_1']+datapre['period_2']+datapre['period_3']+datapre['period_4'])/4
datapre['sqr_median']=np.sqrt(datapre['medan'])
datapre['sqr_sum']=np.sqrt(datapre['sum'])

In [ ]:
X = datapre.drop(['Analytical thinking','Systemic thinking','Adaptability', 'Focus','id'], axis = 1)
pred_Analytical_thinking = rf_model_Analytical_thinking.predict(X)
pred_Analytical_thinking

In [ ]:
X = datapre.drop(['Analytical thinking','Systemic thinking','Adaptability', 'Focus','id'], axis = 1)
X['Analytical_thinking']=pred_Analytical_thinking
pred_Systemic_thinking = rf_model_Systemic_thinking.predict(X)
pred_Systemic_thinking

In [ ]:
X = datapre.drop(['Adaptability', 'Focus','id'], axis = 1)
X['Systemic thinking']=pred_Systemic_thinking
pred_Adaptability = rf_model_Adaptability.predict(X)
pred_Adaptability

In [ ]:
X = datapre.drop(['Focus','id'], axis = 1)
X['Adaptability']=pred_Adaptability
pred_Focus = rf_model_Focus.predict(X)
len(pred_Focus)

In [ ]:
datapre=pd.read_csv('sample_solution.csv')
datapre  

In [ ]:
datapre['Analytical thinking']=pred_Analytical_thinking
datapre['Systemic thinking']=pred_Systemic_thinking
datapre['Adaptability']=pred_Adaptability
datapre['Focus']=pred_Focus

In [ ]:
datapre.info()

In [ ]:
datapre.to_csv('dataprebs5.csv',index=False)

In [ ]:
dataprev=pd.read_csv('dataprebs5.csv')

In [ ]:
dataprev['Adaptability'].value_counts()

In [ ]:
dataprev['Analytical thinking'].value_counts()

In [ ]:
dataprev['Focus'].value_counts()

In [ ]:
dataprev['Systemic thinking'].value_counts()

In [ ]:
datapre.info()